In [14]:
import os
import torch
import cv2
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from src.utils.plotting import make_matching_figure
from src.loftr import LoFTR, default_cfg
import preprocessing as prep
import validation

input_dir = '../../data/train/' # directory of the training data

In [2]:
all_scenes = prep.get_scenes(input_dir)
all_scenes

['brandenburg_gate',
 'british_museum',
 'buckingham_palace',
 'colosseum_exterior',
 'grand_place_brussels',
 'lincoln_memorial_statue',
 'notre_dame_front_facade',
 'pantheon_exterior',
 'piazza_san_marco',
 'sacre_coeur',
 'sagrada_familia',
 'st_pauls_cathedral',
 'st_peters_square',
 'taj_mahal',
 'temple_nara_japan',
 'trevi_fountain']

In [6]:
scenes = [all_scenes[0]]

In [7]:
pairs = prep.load_pairs(scenes,input_dir)

loading category 1 of 1: brandenburg_gate


In [16]:
pairs

,pair,covisibility,fundamental_matrix,scene
0,90920828_5082887495-20133057_3035445116,0.905,-9.27158516e-04 1.22034601e-01 -9.87725452e+01...,brandenburg_gate
1,90920828_5082887495-17262282_1141017004,0.936,1.68899800e-03 1.18269247e-01 -1.02939518e+02 ...,brandenburg_gate
2,20133057_3035445116-17262282_1141017004,0.908,-5.56308440e-04 1.74170978e-02 -2.49391042e+01...,brandenburg_gate
3,38600512_2168650655-17262282_1141017004,0.901,1.26837683e-02 -5.42924979e-01 4.29820975e+02 ...,brandenburg_gate
4,60770994_853214983-17262282_1141017004,0.912,-1.05214974e-02 -4.87393290e-01 3.90468870e+02...,brandenburg_gate
...,...,...,...,...
61070,67494650_6905234882-63229395_2704362565,0.019,-1.83952792e-02 1.11447290e-01 -8.55666688e+01...,brandenburg_gate
61071,67494650_6905234882-65128211_1900926684,0.037,-3.19883341e-02 5.26823598e-01 -3.14236536e+02...,brandenburg_gate
61072,67494650_6905234882-65235481_5082884383,0.095,-1.39080165e-02 7.79098666e-01 -4.67854880e+02...,brandenburg_gate
61073,67494650_6905234882-67080033_7235561554,0.000,9.69361910e-02 -1.65968509e+00 9.28898161e+02 ...,brandenburg_gate


In [9]:
# Initialize LoFTR
matcher = LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.eval()

In [19]:
fund_matrix_list = []
pair_id_list = []

for index, row in pairs.head(5).iterrows():
    
    split_pair = pairs.pair[index].split('-')
    img_id0 = split_pair[0]
    img_id1 = split_pair[1]
    
    img0_pth = os.path.join(input_dir, pairs.scene[index], "images", str(img_id0 + '.jpg'))
    img1_pth = os.path.join(input_dir, pairs.scene[index], "images", str(img_id1 + '.jpg'))
    img0_raw = cv2.imread(img0_pth, cv2.IMREAD_GRAYSCALE)
    img1_raw = cv2.imread(img1_pth, cv2.IMREAD_GRAYSCALE)
    img0_raw = cv2.resize(img0_raw, (640, 480))
    img1_raw = cv2.resize(img1_raw, (640, 480))

    img0 = torch.from_numpy(img0_raw)[None][None] / 255.
    img1 = torch.from_numpy(img1_raw)[None][None] / 255.
    batch = {'image0': img0, 'image1': img1}
    
    with torch.no_grad():
        matcher(batch)
        mkpts0 = batch['mkpts0_f'].cpu().numpy()
        mkpts1 = batch['mkpts1_f'].cpu().numpy()
        mconf = batch['mconf'].cpu().numpy()
        
    F = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.1, 0.99, 20000)
    
    fund_matrix_list.append(" ".join(str(num) for num in F[0].flatten().tolist()))
    pair_id_list.append(";".join(["phototourism",pairs.scene[index],pairs.pair[index]]))
    
maa = validation.evaluate(input_dir, pair_id_list, fund_matrix_list)
print(f'mAA={maa:.05f} (n={len(pair_id_list)})')

mAA=0.46000 (n=5)


ToDo: 
- add IF CPU/GPU
- save mkpts and mconf, so it can be used for drawing later

In [ ]:
# Draw
color = cm.jet(mconf)
text = [
    'LoFTR',
    'Matches: {}'.format(len(mkpts0)),
]
fig = make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, text=text)

## LoFTR example on single image pair

In [ ]:
# Initialize LoFTR and load two images
# matcher.eval() will notify all your layers that you are in eval mode, that way, batchnorm or dropout layers will work in eval mode instead of training mode.
matcher = LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.eval()

img0_pth = "../../data/test_images/1cf87530/0143f47ee9e54243a1b8454f3e91621a.png"
img1_pth = "../../data/test_images/1cf87530/a5a9975574c94ff9a285f58c39b53d2c.png"
img0_raw = cv2.imread(img0_pth, cv2.IMREAD_GRAYSCALE)
img1_raw = cv2.imread(img1_pth, cv2.IMREAD_GRAYSCALE)
img0_raw = cv2.resize(img0_raw, (640, 480))
img1_raw = cv2.resize(img1_raw, (640, 480))

img0 = torch.from_numpy(img0_raw)[None][None] / 255.
img1 = torch.from_numpy(img1_raw)[None][None] / 255.
batch = {'image0': img0, 'image1': img1}

In [ ]:
# Inference with LoFTR and get prediction
# torch.no_grad() impacts the autograd engine and deactivate it. It will reduce memory usage and speed up computations but you won’t be able to backprop (which you don’t want in an eval script).
with torch.no_grad():
    matcher(batch)
    mkpts0 = batch['mkpts0_f'].cpu().numpy()
    mkpts1 = batch['mkpts1_f'].cpu().numpy()
    mconf = batch['mconf'].cpu().numpy()

In [ ]:
# Draw
color = cm.jet(mconf)
text = [
    'LoFTR',
    'Matches: {}'.format(len(mkpts0)),
]
fig = make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, text=text)